In [1]:
setwd('/home/thlamp/tcga/data')
library(TCGAbiolinks)
library(survminer)
library(DESeq2)
library(MOFA2)
library(MOFAdata)

Loading required package: ggplot2

Loading required package: ggpubr

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading require

# Find complete samples

In [2]:
dataPrep <- readRDS("downloaded_dataPrep.rds")
dataPrep

class: RangedSummarizedExperiment 
dim: 60660 421 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(60660): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000288674.1 ENSG00000288675.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(421): TCGA-FV-A3I0-01A-11R-A22L-07
  TCGA-DD-A3A6-11A-11R-A22L-07 ... TCGA-CC-A5UD-01A-11R-A28V-07
  TCGA-BC-A10W-11A-11R-A131-07
colData names(64): barcode patient ... released sample.aux

In [3]:
dataAssy.miR <- readRDS("downloaded_dataAssy_miR.rds")

## Take columns that start with "read_count_" (for raw data) and create matrix

In [4]:
read_countData <-  colnames(dataAssy.miR)[grep("read_count_", colnames(dataAssy.miR))]

data_matrix_miR <- dataAssy.miR[,read_countData]

colnames(data_matrix_miR) <- gsub("read_count_","", colnames(data_matrix_miR))

head(data_matrix_miR, 2)

,TCGA-FV-A3I0-01A-11R-A22J-13,TCGA-ES-A2HS-01A-11R-A17X-13,TCGA-UB-A7MA-01A-11R-A343-13,TCGA-DD-A4NV-01A-11R-A310-13,TCGA-T1-A6J8-01A-11R-A32J-13,TCGA-DD-AADF-01A-11R-A41H-13,TCGA-DD-AACP-01A-11R-A41H-13,TCGA-CC-5261-01A-01R-A130-13,TCGA-DD-A1EC-11A-11R-A130-13,TCGA-DD-AADN-01A-11R-A41H-13,⋯,TCGA-DD-AADY-01A-11R-A41H-13,TCGA-CC-A7IH-01A-11R-A33A-13,TCGA-CC-A7IJ-01A-11R-A343-13,TCGA-G3-AAV1-01A-11R-A38M-13,TCGA-BC-A10X-01A-11R-A130-13,TCGA-G3-A25Z-01A-11R-A16S-13,TCGA-DD-A3A2-11A-11R-A214-13,TCGA-DD-A3A2-01A-11R-A214-13,TCGA-ZS-A9CG-01A-11R-A37G-13,TCGA-ZP-A9D0-01A-11R-A37G-13
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
hsa-let-7a-1,68731,16966,44411,41938,59698,30805,26385,141609,100036,12907,⋯,46125,23894,102617,22193,75118,57368,28422,10481,31753,16776
hsa-let-7a-2,68386,17161,44620,41873,59753,30638,26314,140961,99931,12863,⋯,46136,23652,103224,22293,75366,57330,28642,10356,31748,16823


In [5]:
data_met <- readRDS("downloaded_data_met.rds")
data_met

class: RangedSummarizedExperiment 
dim: 485577 427 
metadata(1): data_release
assays(1): ''
rownames(485577): cg13869341 cg14008030 ... cg11478607 cg08417382
rowData names(52): address_A address_B ... MASK_extBase MASK_general
colnames(427): TCGA-BC-A110-11A-11D-A132-05
  TCGA-G3-A3CG-01A-11D-A20Z-05 ... TCGA-ZS-A9CG-01A-11D-A36Y-05
  TCGA-ZP-A9D0-01A-11D-A36Y-05
colData names(64): barcode patient ... released sample.aux

In [ ]:
mRNASamples <- colData(dataPrep)$sample

miRNASamples <- colnames(data_matrix_miR)
# keep first 16 characters from colnames, which are sample ids
miRNASamples <- substr(miRNASamples, 0, 16)
 
methylationSamples <- colData(data_met)$sample

In [7]:
completeSamples <- Reduce(intersect, list(mRNASamples, miRNASamples, methylationSamples))
length(completeSamples)

[1] 408

# mRNA

In [8]:
dataPrep <- readRDS("downloaded_dataPrep.rds")
dataPrep

class: RangedSummarizedExperiment 
dim: 60660 421 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(60660): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000288674.1 ENSG00000288675.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(421): TCGA-FV-A3I0-01A-11R-A22L-07
  TCGA-DD-A3A6-11A-11R-A22L-07 ... TCGA-CC-A5UD-01A-11R-A28V-07
  TCGA-BC-A10W-11A-11R-A131-07
colData names(64): barcode patient ... released sample.aux

## Filtering for duplicated samples

In [9]:
# duplicates_index_to_remove <- function(patient,input_matrix) {
#   # for every element in patient search colnames that start with this sample id, whih are from duplicated samples 
#   duplicated_columns <- colnames(input_matrix)[grep(patient, colnames(input_matrix))]
    
#   # Count zeros in the specified columns
#   zero_counts <- colSums(input_matrix[, duplicated_columns] == 0)
#   print(zero_counts)

#   # find which column for every duplicated sample has more zeros
#   index_of_larger_number <- which.max(zero_counts)

#   # Find the index from the matrix, of the column with more zeros to remove
#   column_index_to_remove <- which(colnames(input_matrix) %in% duplicated_columns[index_of_larger_number])

#   return (column_index_to_remove)
# }

In [10]:
# # find duplicated samples in colData(dataPrep)
# patient_duplicates <- colData(dataPrep)$sample[which(duplicated(colData(dataPrep)$sample))]

# # use function duplicates_index_to_remove for every element in patient_duplicates
# patient_duplicates_to_remove <- sapply(patient_duplicates, duplicates_index_to_remove, input_matrix = assay(dataPrep,"unstranded"))

# print("Index of duplicates with the most zeros per patient with duplicates")
# patient_duplicates_to_remove

# print("The names of the columns that will be removed")
# colnames(dataPrep)[patient_duplicates_to_remove]
# samples_to_remove <- colnames(dataPrep)[patient_duplicates_to_remove]

In [11]:
# # Remove the specified-duplicated columns
# dataPrep <- dataPrep[, which(!colnames(dataPrep) %in% samples_to_remove)]
# dataPrep

In [12]:
# length(colData(dataPrep)$sample)
# which(duplicated(colData(dataPrep)$sample))

In [13]:
# which(colData(dataPrep)$barcode %in% c('TCGA-BL-A13J-01A-11R-A10U-07','TCGA-BL-A13I-01A-11R-A277-07','TCGA-BL-A0C8-01A-11R-A10U-07'))
# which(colnames(dataPrep) %in% c('TCGA-BL-A13J-01A-11R-A10U-07','TCGA-BL-A13I-01A-11R-A277-07','TCGA-BL-A0C8-01A-11R-A10U-07'))

## Filtering for complete samples

In [14]:
# # Remove the specified-duplicated columns
dataPrep <- dataPrep[, which(colData(dataPrep)$sample %in% completeSamples)]
dataPrep

class: RangedSummarizedExperiment 
dim: 60660 408 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(60660): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000288674.1 ENSG00000288675.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(408): TCGA-FV-A3I0-01A-11R-A22L-07
  TCGA-DD-A3A6-01A-11R-A22L-07 ... TCGA-CC-A5UD-01A-11R-A28V-07
  TCGA-BC-A10W-11A-11R-A131-07
colData names(64): barcode patient ... released sample.aux

In [15]:
colnames(dataPrep) <- colData(dataPrep)$sample

## Filtering selected features 

In [16]:
selectedFeatures<- read.csv("/home/thlamp/tcga/data/DEGs450.csv")
head(selectedFeatures, 3)

,x
,<chr>
1,ENSG00000160339.16
2,ENSG00000019169.10
3,ENSG00000136011.15


In [17]:
dataPrep <- dataPrep[which(rownames(dataPrep) %in% selectedFeatures$x),]
dataPrep

class: RangedSummarizedExperiment 
dim: 150 408 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(150): ENSG00000007350.17 ENSG00000019169.10 ...
  ENSG00000273777.5 ENSG00000286403.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(408): TCGA-FV-A3I0-01A TCGA-DD-A3A6-01A ... TCGA-CC-A5UD-01A
  TCGA-BC-A10W-11A
colData names(64): barcode patient ... released sample.aux

## State MOFA model

In [18]:
# Create a DESeqDataSet object
dds <- DESeqDataSetFromMatrix(countData = assay(dataPrep,"unstranded"),
                              colData = colData(dataPrep),
                              design = ~ 1)


# Estimate size factors
dds <- estimateSizeFactors(dds)

# Perform variance-stabilizing transformation with design information
vsd <- varianceStabilizingTransformation(dds, blind = TRUE)

# Extract the normalized counts
normalized_counts <- assay(vsd)

head(normalized_counts, 2)

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



,TCGA-FV-A3I0-01A,TCGA-DD-A3A6-01A,TCGA-BD-A3ER-01A,TCGA-CC-5261-01A,TCGA-DD-AAVZ-01A,TCGA-DD-AADN-01A,TCGA-DD-A1EB-01A,TCGA-LG-A9QD-01A,TCGA-G3-A25T-01A,TCGA-XR-A8TE-01A,⋯,TCGA-FV-A2QR-01A,TCGA-DD-AADY-01A,TCGA-BW-A5NO-01A,TCGA-CC-A7IJ-01A,TCGA-2Y-A9H8-01A,TCGA-BC-A10X-01A,TCGA-DD-A3A2-11A,TCGA-DD-A3A2-01A,TCGA-CC-A5UD-01A,TCGA-BC-A10W-11A
ENSG00000007350.17,-2.005792,3.668439,0.008955007,-0.7072536,-2.744387,-2.530052,-0.7269816,0.1496377,-2.037746,-1.049837,⋯,-2.237870,-0.04683096,0.3309628,-3.070753,-3.202897,3.337705,3.283273,0.6638941,-3.828411,3.483122
ENSG00000019169.10,3.770181,4.862798,3.655328409,0.4305947,-5.128186,5.235758,4.8880551,2.8254859,5.331767,6.245451,⋯,4.688055,1.03912578,1.7541576,-1.180819,3.241514,12.954035,16.244889,13.3355976,-1.905900,14.414473


### check data

In [25]:
# check order of colnames in matrix and colData
identical(colnames(normalized_counts), colData(dataPrep)$barcode)

[1] TRUE

In [26]:
# find index of states by grep in colnames
x<-grep("-11A-", colnames(normalized_counts))
y<-grep("-11A-", colnames(normalized_counts), invert=TRUE)

# compare index of states by grep in colnames with index of NT/TP in colData
identical(x, which(colData(dataPrep)$shortLetterCode %in% "NT"))
identical(y, which(colData(dataPrep)$shortLetterCode %in% "TP"))

[1] TRUE

[1] TRUE

## Tumor stage MOFA model

In [19]:
dataPrep <- dataPrep[,which(colData(dataPrep)$shortLetterCode %in% "TP")]
dim(dataPrep)
dataPrep <- dataPrep[,which(!colData(dataPrep)$ajcc_pathologic_stage  %in% NA)]
dim(dataPrep)

[1] 2000  406

[1] 2000  404

In [20]:
# Create a DESeqDataSet object
dds <- DESeqDataSetFromMatrix(countData = assay(dataPrep,"unstranded"),
                              colData = colData(dataPrep),
                              design = ~ 1)

# Estimate size factors
dds <- estimateSizeFactors(dds)

# Perform variance-stabilizing transformation with design information
vsd <- varianceStabilizingTransformation(dds, blind = TRUE)

# Extract the normalized counts
normalized_counts <- assay(vsd)

head(normalized_counts, 2)

,TCGA-CU-A3KJ-01A,TCGA-K4-A3WU-01B,TCGA-DK-A3IU-01A,TCGA-GV-A40G-01A,TCGA-DK-A3IN-01A,TCGA-SY-A9G0-01A,TCGA-XF-A9SU-01A,TCGA-UY-A78O-01A,TCGA-GD-A2C5-01A,TCGA-G2-A2EK-01A,⋯,TCGA-DK-A1AD-01A,TCGA-XF-AAMJ-01A,TCGA-DK-A1A6-01A,TCGA-XF-AAN2-01A,TCGA-GU-A42Q-01A,TCGA-GC-A4ZW-01A,TCGA-XF-AAMY-01A,TCGA-XF-AAN7-01A,TCGA-E7-A6MD-01A,TCGA-ZF-A9R5-01A
ENSG00000000003.15,11.76353,14.29300,10.64870,13.33954,12.36204,11.76163,12.31894,13.56799,13.28282,14.08727,⋯,14.42904,12.57242,13.27070,11.86477,10.89763,13.09511,13.12161,12.17689,12.60601,14.33528
ENSG00000000419.13,11.90282,11.49734,11.64532,11.81922,10.96108,11.63830,11.38279,11.94002,11.63557,11.88206,⋯,11.64496,11.34744,12.44894,11.78058,11.56383,11.78028,11.36586,11.12770,11.21504,11.17671


### check data

## MOFA mRNA

In [11]:
sample <- colnames(normalized_counts)

shortLetterCode <- ""
shortLetterCode <- apply(as.matrix(sample),1, function (x){dataPrep$shortLetterCode[which(dataPrep$sample %in% x)]})

ajcc_pathologic_stage <- ""
ajcc_pathologic_stage <- apply(as.matrix(sample),1, function (x){dataPrep$ajcc_pathologic_stage[which(dataPrep$sample %in% x)]})

age_at_diagnosis <- ""
age_at_diagnosis <- apply(as.matrix(sample),1, function (x){dataPrep$age_at_diagnosis[which(dataPrep$sample %in% x)]})

primary_diagnosis <- ""
primary_diagnosis <- apply(as.matrix(sample),1, function (x){dataPrep$primary_diagnosis[which(dataPrep$sample %in% x)]})

cigarettes_per_day  <- ""
cigarettes_per_day  <- apply(as.matrix(sample),1, function (x){dataPrep$cigarettes_per_day [which(dataPrep$sample %in% x)]})

gender <- ""
gender <- apply(as.matrix(sample),1, function (x){dataPrep$gender[which(dataPrep$sample %in% x)]})

ethnicity <- ""
ethnicity <- apply(as.matrix(sample),1, function (x){dataPrep$ethnicity[which(dataPrep$sample %in% x)]})

age_at_index <- ""
age_at_index <- apply(as.matrix(sample),1, function (x){dataPrep$age_at_index[which(dataPrep$sample %in% x)]})

race <- ""
race <- apply(as.matrix(sample),1, function (x){dataPrep$race[which(dataPrep$sample %in% x)]})

days_to_death <- ""
days_to_death <- apply(as.matrix(sample),1, function (x){dataPrep$days_to_death[which(dataPrep$sample %in% x)]})

mrna_samples <- data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, cigarettes_per_day, gender,ethnicity, 
                           age_at_index, race, days_to_death)
head(mrna_samples,2)

ERROR: Error in data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, : arguments imply differing number of rows: 421, 0


### Biomart transcriptIDs to ensemblIDs

In [116]:
library(biomaRt)

In [117]:
ensembl <- useEnsembl(biomart = 'genes',dataset = "hsapiens_gene_ensembl",version = 102)

#### Search for unique transcriptIDs to avoid repetitions of same transcriptIDs

In [118]:
# check number of rownames to be sure there are all in bm_results with ensembl_gene_id
length(rownames(normalized_counts))

[1] 1000

In [119]:
searchAttributes(mart = ensembl, pattern = "ensembl")

,name,description,page
,<chr>,<chr>,<chr>
1,ensembl_gene_id,Gene stable ID,feature_page
2,ensembl_gene_id_version,Gene stable ID version,feature_page
3,ensembl_transcript_id,Transcript stable ID,feature_page
4,ensembl_transcript_id_version,Transcript stable ID version,feature_page
5,ensembl_peptide_id,Protein stable ID,feature_page
6,ensembl_peptide_id_version,Protein stable ID version,feature_page
7,ensembl_exon_id,Exon stable ID,feature_page
51,clone_based_ensembl_gene,Clone-based (Ensembl) gene ID,feature_page
52,clone_based_ensembl_transcript,Clone-based (Ensembl) transcript ID,feature_page


In [120]:
gene_ids_versions <- rownames(normalized_counts)

bm_results <- getBM(attributes = c('ensembl_gene_id', 'ensembl_gene_id_version'),
      filters = 'ensembl_gene_id_version',
      values = gene_ids_versions, 
      mart = ensembl)

In [121]:
dim(bm_results)
head(bm_results,2)

[1] 1000    2

,ensembl_gene_id,ensembl_gene_id_version
,<chr>,<chr>
1,ENSG00000002586,ENSG00000002586.20
2,ENSG00000005022,ENSG00000005022.6


In [122]:
# Load the tidyverse package
library(tidyverse)

# Create the initial data frame
df <- data.frame(
  ensembl_gene_id_version = gene_ids_versions[grep("PAR_Y", gene_ids_versions)]
)

# Use regular expressions to extract the desired part and add it as a new column
df <- df %>%
  mutate(
    ensembl_gene_id = str_remove(ensembl_gene_id_version, "\\.\\d+")
  )

# Reorder the columns of df to match bm_results
df <- df[, colnames(bm_results)]

# Combine the two data frames using rbind
bm_results <- rbind(bm_results, df)

dim(bm_results)

[1] 1000    2

#### Match ensembl_gene_id with the data from mapping_data by ensembl_transcript_id

In [123]:
rownames(normalized_counts) <- apply(as.matrix(rownames(normalized_counts)),1, function (x){bm_results$ensembl_gene_id[which(bm_results$ensembl_gene_id_version %in% x)]})
head(normalized_counts, 2)

,TCGA-CU-A3KJ-01A,TCGA-K4-A3WU-01B,TCGA-DK-A3IU-01A,TCGA-GV-A40G-01A,TCGA-DK-A3IN-01A,TCGA-SY-A9G0-01A,TCGA-XF-A9SU-01A,TCGA-UY-A78O-01A,TCGA-GD-A2C5-01A,TCGA-G2-A2EK-01A,⋯,TCGA-DK-A1AD-01A,TCGA-XF-AAMJ-01A,TCGA-DK-A1A6-01A,TCGA-XF-AAN2-01A,TCGA-GU-A42Q-01A,TCGA-GC-A4ZW-01A,TCGA-XF-AAMY-01A,TCGA-XF-AAN7-01A,TCGA-E7-A6MD-01A,TCGA-ZF-A9R5-01A
ENSG00000002586,14.52979,13.98095,13.46209,14.40897,13.05310,13.8249,14.07211,13.15975,12.96384,12.86549,⋯,11.67596,14.15895,13.16802,13.01508,12.61082,14.03964,13.40969,14.91011,13.03294,11.73564
ENSG00000005022,14.03737,13.12509,12.89316,13.78490,13.56822,12.8805,13.29391,13.56997,13.73354,13.52359,⋯,13.73161,13.37535,14.01804,14.01231,13.73887,15.59624,12.65848,13.90818,14.05366,13.64436


#### check gene names

In [124]:
test_df <- data.frame(gene_ids=gene_ids_versions)
head(test_df)

,gene_ids
,<chr>
1,ENSG00000000003.15
2,ENSG00000000419.13
3,ENSG00000001617.12
4,ENSG00000002549.12
5,ENSG00000002586.20
6,ENSG00000002834.18


In [125]:
test_df$modified_gene_ids <- test_df$gene_ids
test_df$modified_gene_ids <- gsub("\\..*$", "", test_df$modified_gene_ids)
head(test_df)

,gene_ids,modified_gene_ids
,<chr>,<chr>
1,ENSG00000000003.15,ENSG00000000003
2,ENSG00000000419.13,ENSG00000000419
3,ENSG00000001617.12,ENSG00000001617
4,ENSG00000002549.12,ENSG00000002549
5,ENSG00000002586.20,ENSG00000002586
6,ENSG00000002834.18,ENSG00000002834


In [126]:
identical(rownames(normalized_counts), test_df$modified_gene_ids)

[1] TRUE

In [97]:
# check order of colnames in matrix and colData
identical(colnames(normalized_counts), colData(dataPrep)$barcode)

[1] TRUE

In [98]:
# find index of states by grep in colnames
x<-grep("-11A-", colnames(normalized_counts))
y<-grep("-11A-", colnames(normalized_counts), invert=TRUE)

# compare index of states by grep in colnames with index of NT/TP in colData
identical(x, which(colData(dataPrep)$shortLetterCode %in% "NT"))
identical(y, which(colData(dataPrep)$shortLetterCode %in% "TP"))

[1] TRUE

[1] TRUE

# miRNA

In [19]:
dataAssy.miR <- readRDS("downloaded_dataAssy_miR.rds")

## Take columns that start with "read_count_" (for raw data) and create matrix

In [20]:
read_countData <-  colnames(dataAssy.miR)[grep("read_count_", colnames(dataAssy.miR))]

data_matrix_miR <- dataAssy.miR[,read_countData]

colnames(data_matrix_miR) <- gsub("read_count_","", colnames(data_matrix_miR))

head(data_matrix_miR, 2)

,TCGA-FV-A3I0-01A-11R-A22J-13,TCGA-ES-A2HS-01A-11R-A17X-13,TCGA-UB-A7MA-01A-11R-A343-13,TCGA-DD-A4NV-01A-11R-A310-13,TCGA-T1-A6J8-01A-11R-A32J-13,TCGA-DD-AADF-01A-11R-A41H-13,TCGA-DD-AACP-01A-11R-A41H-13,TCGA-CC-5261-01A-01R-A130-13,TCGA-DD-A1EC-11A-11R-A130-13,TCGA-DD-AADN-01A-11R-A41H-13,⋯,TCGA-DD-AADY-01A-11R-A41H-13,TCGA-CC-A7IH-01A-11R-A33A-13,TCGA-CC-A7IJ-01A-11R-A343-13,TCGA-G3-AAV1-01A-11R-A38M-13,TCGA-BC-A10X-01A-11R-A130-13,TCGA-G3-A25Z-01A-11R-A16S-13,TCGA-DD-A3A2-11A-11R-A214-13,TCGA-DD-A3A2-01A-11R-A214-13,TCGA-ZS-A9CG-01A-11R-A37G-13,TCGA-ZP-A9D0-01A-11R-A37G-13
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
hsa-let-7a-1,68731,16966,44411,41938,59698,30805,26385,141609,100036,12907,⋯,46125,23894,102617,22193,75118,57368,28422,10481,31753,16776
hsa-let-7a-2,68386,17161,44620,41873,59753,30638,26314,140961,99931,12863,⋯,46136,23652,103224,22293,75366,57330,28642,10356,31748,16823


## Filtering for duplicated samples

In [21]:
# duplicates_index_to_remove <- function(patient,input_matrix) {
#   # for every element in patient search colnames that start with this sample id, whih are from duplicated samples 
#   duplicated_columns <- colnames(input_matrix)[grep(patient, colnames(input_matrix))]
    
#   # Count zeros in the specified columns
#   zero_counts <- colSums(input_matrix[, duplicated_columns] == 0)
#   print(zero_counts)

#   # find which column for every duplicated sample has more zeros
#   index_of_larger_number <- which.max(zero_counts)

#   # Find the index from the matrix, of the column with more zeros to remove
#   column_index_to_remove <- which(colnames(input_matrix) %in% duplicated_columns[index_of_larger_number])

#   return (column_index_to_remove)
# }

In [22]:
# # keep first 16 characters from colnames, which are sample ids
# sample_ids <- substr(colnames(data_matrix_miR), 0, 16)

# # find duplicated samples in colData(dataPrep)
# patient_duplicates <- sample_ids[which(duplicated(sample_ids))]

# # use function duplicates_index_to_remove for every element in patient_duplicates
# patient_duplicates_to_remove <- sapply(patient_duplicates, duplicates_index_to_remove, input_matrix = data_matrix_miR)

# print("Index of duplicates with the most zeros per patient with duplicates")
# patient_duplicates_to_remove

# print("The names of the columns that will be removed")
# colnames(data_matrix_miR)[patient_duplicates_to_remove]
# samples_to_remove <- colnames(data_matrix_miR)[patient_duplicates_to_remove]

In [23]:
# # Remove the specified-duplicated columns
# data_matrix_miR <- data_matrix_miR[, which(!colnames(data_matrix_miR) %in% samples_to_remove)]
# dim(data_matrix_miR)

## Filtering for complete samples

In [24]:
# keep first 16 characters from colnames, which are sample ids
colnames(data_matrix_miR) <- substr(colnames(data_matrix_miR), 0, 16)

# Remove the specified-duplicated columns
data_matrix_miR <- data_matrix_miR[, which(colnames(data_matrix_miR) %in% completeSamples)]
dim(data_matrix_miR)

[1] 1881  408

## Filtering selected features 

In [25]:
selectedFeatures<- read.csv("/home/thlamp/tcga/data/DEGs450.csv")
head(selectedFeatures, 3)

,x
,<chr>
1,ENSG00000160339.16
2,ENSG00000019169.10
3,ENSG00000136011.15


In [26]:
data_matrix_miR <- data_matrix_miR[which(rownames(data_matrix_miR) %in% selectedFeatures$x),]
dim(data_matrix_miR)

[1] 150 408

## Prepare meatadata for MOFA

In [27]:
dataPrep <- readRDS("downloaded_dataPrep.rds")
data_met <- readRDS("downloaded_data_met.rds")

In [28]:
mrna_coldata <- colData(dataPrep)[, c("sample", "shortLetterCode", "ajcc_pathologic_stage")]
head(mrna_coldata, 2)
meth_coldata <- colData(data_met)[, c("sample", "shortLetterCode", "ajcc_pathologic_stage")]
head(meth_coldata, 2)

DataFrame with 2 rows and 3 columns
                                       sample shortLetterCode
                                  <character>     <character>
TCGA-FV-A3I0-01A-11R-A22L-07 TCGA-FV-A3I0-01A              TP
TCGA-DD-A3A6-11A-11R-A22L-07 TCGA-DD-A3A6-11A              NT
                             ajcc_pathologic_stage
                                       <character>
TCGA-FV-A3I0-01A-11R-A22L-07              Stage II
TCGA-DD-A3A6-11A-11R-A22L-07              Stage II

DataFrame with 2 rows and 3 columns
                                       sample shortLetterCode
                                  <character>     <character>
TCGA-BC-A110-11A-11D-A132-05 TCGA-BC-A110-11A              NT
TCGA-G3-A3CG-01A-11D-A20Z-05 TCGA-G3-A3CG-01A              TP
                             ajcc_pathologic_stage
                                       <character>
TCGA-BC-A110-11A-11D-A132-05                    NA
TCGA-G3-A3CG-01A-11D-A20Z-05               Stage I

In [29]:
mirna_coldata <- rbind(mrna_coldata, meth_coldata)
dim(mirna_coldata)

[1] 848   3

In [30]:
mirna_coldata <- mirna_coldata[!duplicated(mirna_coldata[,"sample"]),]
dim(mirna_coldata)

[1] 436   3

In [31]:
miRcolData <- data.frame(sample=colnames(data_matrix_miR))
head(miRcolData,2)

,sample
,<chr>
1,TCGA-FV-A3I0-01A
2,TCGA-ES-A2HS-01A


In [32]:
# miRcolData$sample <- ""
# miRcolData$sample <- substr(miRcolData$barcode, 0, 16)
# head(miRcolData,2)

In [33]:
miRcolData$shortLetterCode <- ""
miRcolData$shortLetterCode <- apply(as.matrix(miRcolData$sample),1, function (x){mirna_coldata$shortLetterCode[which(mirna_coldata$sample %in% x)]})

miRcolData$ajcc_pathologic_stage <- ""
miRcolData$ajcc_pathologic_stage <- apply(as.matrix(miRcolData$sample),1, function (x){mirna_coldata$ajcc_pathologic_stage[which(mirna_coldata$sample %in% x)]})

dim(miRcolData)
head(miRcolData,2)

[1] 408   3

,sample,shortLetterCode,ajcc_pathologic_stage
,<chr>,<chr>,<chr>
1,TCGA-FV-A3I0-01A,TP,Stage II
2,TCGA-ES-A2HS-01A,TP,Stage I


## State MOFA model

In [34]:
identical(colnames(data_matrix_miR), miRcolData$sample)

[1] TRUE

In [35]:
# Create a DESeqDataSet object
miRdds <- DESeqDataSetFromMatrix(countData = data_matrix_miR,
                              colData = miRcolData,
                              design = ~1)

# Estimate size factors
miRdds <- estimateSizeFactors(miRdds)

miRvst <- varianceStabilizingTransformation(miRdds, blind = TRUE)

# Extract the normalized counts
normalized_counts_miR <- assay(miRvst)

head(normalized_counts_miR, 2)

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



,TCGA-FV-A3I0-01A,TCGA-ES-A2HS-01A,TCGA-UB-A7MA-01A,TCGA-DD-A4NV-01A,TCGA-T1-A6J8-01A,TCGA-DD-AADF-01A,TCGA-DD-AACP-01A,TCGA-CC-5261-01A,TCGA-DD-A1EC-11A,TCGA-DD-AADN-01A,⋯,TCGA-DD-AADY-01A,TCGA-CC-A7IH-01A,TCGA-CC-A7IJ-01A,TCGA-G3-AAV1-01A,TCGA-BC-A10X-01A,TCGA-G3-A25Z-01A,TCGA-DD-A3A2-11A,TCGA-DD-A3A2-01A,TCGA-ZS-A9CG-01A,TCGA-ZP-A9D0-01A
hsa-let-7c,12.01826,14.68580,12.07306,14.28009,13.80003,13.32884,12.14940,13.16593,14.79799,11.91559,⋯,12.00031,14.05701,14.73196,13.02025,14.86466,13.67965,14.30978,14.73234,14.37286,13.47230
hsa-mir-101-1,14.66983,16.88868,14.19257,15.60742,15.27675,15.90613,14.27829,15.17924,16.52022,14.38930,⋯,16.00567,17.03607,15.06988,16.32449,15.81956,15.52838,16.56375,15.94249,15.44887,15.38138


-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

### check data

In [43]:
# check order of colnames in matrix and colData
identical(colnames(normalized_counts_miR), miRcolData$barcode)

[1] TRUE

In [44]:
# find index of states by grep in colnames
x<-grep("-11A-", colnames(normalized_counts_miR))
y<-grep("-11A-", colnames(normalized_counts_miR), invert=TRUE)

# compare index of states by grep in colnames with index of NT/TP in colData
identical(x, which(miRcolData$shortLetterCode %in% "NT"))
identical(y, which(miRcolData$shortLetterCode %in% "TP"))

[1] TRUE

[1] TRUE

## Tumor stage for MOFA

In [45]:
miRcolData <- miRcolData[which(miRcolData$shortLetterCode %in% "TP"), ]
dim(miRcolData)
miRcolData <- miRcolData[which(!miRcolData$ajcc_pathologic_stage %in% NA), ]
dim(miRcolData)

[1] 406   3

[1] 404   3

In [46]:
data_matrix_miR <- data_matrix_miR[, miRcolData$sample]
dim(data_matrix_miR)
head(data_matrix_miR, 2)

[1] 412 404

,TCGA-DK-A2I2-01A,TCGA-E7-A6MF-01A,TCGA-DK-A3IV-01A,TCGA-FD-A6TA-01A,TCGA-FD-A62S-01A,TCGA-BT-A20T-01A,TCGA-UY-A78M-01A,TCGA-DK-AA6L-01A,TCGA-FD-A5BT-01A,TCGA-KQ-A41S-01A,⋯,TCGA-XF-AAMH-01A,TCGA-GU-A764-01A,TCGA-ZF-A9RE-01A,TCGA-4Z-AA7M-01A,TCGA-GD-A3OQ-01A,TCGA-CU-A3YL-01A,TCGA-FD-A5C0-01A,TCGA-GU-A42R-01A,TCGA-FD-A6TI-01A,TCGA-DK-A6B1-01A
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
hsa-let-7a-1,18587,44934,69295,33473,41076,38391,18145,63904,41545,41340,⋯,30927,28192,28570,55839,19261,24243,17404,46205,77266,90126
hsa-let-7a-2,18617,45179,68297,33315,41241,38219,18167,63826,41446,41514,⋯,30864,28379,28746,55893,19361,23998,17723,45906,76637,89419


In [47]:
identical(colnames(data_matrix_miR), miRcolData$sample)

[1] TRUE

In [48]:
# Create a DESeqDataSet object
miRdds <- DESeqDataSetFromMatrix(countData = data_matrix_miR,
                              colData = miRcolData,
                              design = ~ 1)

# Estimate size factors
miRdds <- estimateSizeFactors(miRdds)

miRvst <- varianceStabilizingTransformation(miRdds, blind = TRUE)

# Extract the normalized counts
normalized_counts_miR <- assay(miRvst)

head(normalized_counts_miR, 2)

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



,TCGA-DK-A2I2-01A,TCGA-E7-A6MF-01A,TCGA-DK-A3IV-01A,TCGA-FD-A6TA-01A,TCGA-FD-A62S-01A,TCGA-BT-A20T-01A,TCGA-UY-A78M-01A,TCGA-DK-AA6L-01A,TCGA-FD-A5BT-01A,TCGA-KQ-A41S-01A,⋯,TCGA-XF-AAMH-01A,TCGA-GU-A764-01A,TCGA-ZF-A9RE-01A,TCGA-4Z-AA7M-01A,TCGA-GD-A3OQ-01A,TCGA-CU-A3YL-01A,TCGA-FD-A5C0-01A,TCGA-GU-A42R-01A,TCGA-FD-A6TI-01A,TCGA-DK-A6B1-01A
hsa-let-7a-1,14.18191,16.95684,15.24064,14.60994,14.66277,15.99551,14.95984,15.72317,14.56781,14.99458,⋯,14.18872,14.74935,15.87080,15.35694,14.78364,14.53320,14.26200,14.61685,15.35964,15.69282
hsa-let-7a-2,14.18422,16.96448,15.22015,14.60320,14.66847,15.98921,14.96155,15.72145,14.56441,15.00053,⋯,14.18580,14.75874,15.87942,15.35830,14.79100,14.51872,14.28798,14.60760,15.34812,15.68175


### check data

In [115]:
# check order of colnames in matrix and colData
identical(colnames(normalized_counts_miR), miRcolData$barcode)

[1] TRUE

In [116]:
# find index of states by grep in colnames
y<-grep("-11A-", colnames(normalized_counts_miR), invert=TRUE)

# compare index of states by grep in colnames with index of NT/TP in colData
identical(y, which(miRcolData$shortLetterCode %in% "TP"))

length(which(miRcolData$ajcc_pathologic_stage %in% NA))

[1] TRUE

[1] 0

# Methylation

In [56]:
# Convertion to M-values for MOFA
# processed_meth_mat_mean_with_na.rds has already removed the duplicated samples
meth_mat_mean <- readRDS("processed_meth_mat_mean.rds")

In [57]:
data_met <- readRDS("downloaded_data_met.rds")
data_met

class: RangedSummarizedExperiment 
dim: 485577 427 
metadata(1): data_release
assays(1): ''
rownames(485577): cg13869341 cg14008030 ... cg11478607 cg08417382
rowData names(52): address_A address_B ... MASK_extBase MASK_general
colnames(427): TCGA-BC-A110-11A-11D-A132-05
  TCGA-G3-A3CG-01A-11D-A20Z-05 ... TCGA-ZS-A9CG-01A-11D-A36Y-05
  TCGA-ZP-A9D0-01A-11D-A36Y-05
colData names(64): barcode patient ... released sample.aux

## Filtering for duplicated samples

In [58]:
# data_met <- data_met[, !duplicated(colData(data_met)$sample)]
# data_met

## Filtering for complete samples

In [59]:
# keep first 16 characters from colnames, which are sample ids
colnames(meth_mat_mean) <- substr(colnames(meth_mat_mean), 0, 16)

# Remove the specified-duplicated columns
meth_mat_mean <- meth_mat_mean[, which(colnames(meth_mat_mean) %in% completeSamples)]
dim(meth_mat_mean)

[1] 34925   408

## Filtering selected features

In [60]:
selectedFeatures<- read.csv("/home/thlamp/tcga/data/DEGs450.csv")
head(selectedFeatures, 3)

,x
,<chr>
1,ENSG00000160339.16
2,ENSG00000019169.10
3,ENSG00000136011.15


In [61]:
meth_mat_mean <- meth_mat_mean[which(rownames(meth_mat_mean) %in% selectedFeatures$x),]
dim(meth_mat_mean)

[1] 150 408

## State for MOFA model

In [62]:
# Convertion to M-values for MOFA
# meth_mat_mean <- readRDS("meth_mat_mean_with_na.rds")

In [63]:
m_values <- log2(meth_mat_mean / (1 - meth_mat_mean))

dim(m_values)
head(m_values, 2)

[1] 150 408

,TCGA-BC-A110-11A,TCGA-G3-A3CG-01A,TCGA-FV-A3I0-01A,TCGA-G3-AAV5-01A,TCGA-EP-A3JL-01A,TCGA-DD-A3A6-01A,TCGA-FV-A3I1-01A,TCGA-KR-A7K2-01A,TCGA-DD-AACP-01A,TCGA-CC-5261-01A,⋯,TCGA-CC-A7IF-01A,TCGA-FV-A2QR-11A,TCGA-DD-A39V-11A,TCGA-G3-A25V-01A,TCGA-CC-A7IH-01A,TCGA-G3-AAV1-01A,TCGA-2Y-A9H9-01A,TCGA-2Y-A9GW-01A,TCGA-ZS-A9CG-01A,TCGA-ZP-A9D0-01A
ENSG00000010319,0.2911843,0.1005068,0.1844318,-0.0919322,-0.03779667,0.7862665,0.09263731,0.1036769,-0.004693073,0.2989035,⋯,0.1032848,0.5127131,0.6326737,-0.08909117,-0.07953877,-0.1893550,-0.7586651,0.08294856,-0.2666402,0.2377414
ENSG00000092607,-0.5778018,0.4870179,2.1962976,0.9571476,0.21194973,-0.9250359,-0.27547691,0.9518820,2.063170224,0.5862042,⋯,1.2039290,-0.8585055,-0.6236350,-0.51489886,0.24861659,0.8909564,0.7931659,0.02110310,-0.1691787,0.5583310


In [46]:
# Create a logical matrix indicating where NAs or NaNs are present
is_na_nan <- is.na(m_values) | is.nan(m_values)

# Summarize each row to check if all elements are NA or NaN
all_na_nan_rows <- apply(is_na_nan, 1, all)

# Identify row indices where all elements are TRUE
rows_with_only_na_nan <- which(all_na_nan_rows)

m_values <- m_values[!all_na_nan_rows, ]
dim(m_values)

[1] 150 408

## Tumor stage for MOFA model

In [55]:
meth_data_stage <- colData(data_met)[which(colData(data_met)$sample  %in% colnames(meth_mat_mean)), c("sample", "shortLetterCode", "ajcc_pathologic_stage")]
dim(meth_data_stage)

[1] 427   3

In [56]:
meth_data_stage <- meth_data_stage[which(meth_data_stage$shortLetterCode %in% "TP"), ]
dim(meth_data_stage)
head(meth_data_stage)
meth_data_stage <- meth_data_stage[which(!meth_data_stage$ajcc_pathologic_stage %in% NA), ]
dim(meth_data_stage)

[1] 377   3

DataFrame with 6 rows and 3 columns
                                       sample shortLetterCode
                                  <character>     <character>
TCGA-G3-A3CG-01A-11D-A20Z-05 TCGA-G3-A3CG-01A              TP
TCGA-FV-A3I0-01A-11D-A22H-05 TCGA-FV-A3I0-01A              TP
TCGA-G3-AAV5-01A-11D-A36Y-05 TCGA-G3-AAV5-01A              TP
TCGA-EP-A3JL-01A-11D-A20Z-05 TCGA-EP-A3JL-01A              TP
TCGA-DD-A3A6-01A-11D-A22H-05 TCGA-DD-A3A6-01A              TP
TCGA-FV-A3I1-01A-11D-A22H-05 TCGA-FV-A3I1-01A              TP
                             ajcc_pathologic_stage
                                       <character>
TCGA-G3-A3CG-01A-11D-A20Z-05               Stage I
TCGA-FV-A3I0-01A-11D-A22H-05              Stage II
TCGA-G3-AAV5-01A-11D-A36Y-05              Stage II
TCGA-EP-A3JL-01A-11D-A20Z-05               Stage I
TCGA-DD-A3A6-01A-11D-A22H-05              Stage II
TCGA-FV-A3I1-01A-11D-A22H-05              Stage II

[1] 353   3

In [57]:
# keep only samlpes that are TP and have tumor stage
meth_mat_mean <- meth_mat_mean[, meth_data_stage$sample]
dim(meth_mat_mean)

[1] 2000  404

In [58]:
m_values <- log2(meth_mat_mean / (1 - meth_mat_mean))

dim(m_values)
head(m_values, 2)

[1] 2000  404

,TCGA-FD-A3NA-01A,TCGA-E7-A6MF-01A,TCGA-UY-A9PB-01A,TCGA-E7-A4IJ-01A,TCGA-DK-A3IV-01A,TCGA-FD-A6TA-01A,TCGA-GU-A42R-01A,TCGA-FD-A6TG-01A,TCGA-UY-A78M-01A,TCGA-FD-A5BT-01A,⋯,TCGA-CF-A7I0-01A,TCGA-E7-A8O7-01A,TCGA-G2-AA3B-01A,TCGA-E7-A7PW-01A,TCGA-CU-A3YL-01A,TCGA-FD-A6TH-01A,TCGA-FD-A5C0-01A,TCGA-BT-A20T-01A,TCGA-FD-A6TI-01A,TCGA-DK-A6B1-01A
ENSG00000021852,-0.4527657,2.768443,-0.2678522,1.180864,-0.223687723,-0.1919902,3.122569,0.06629804,-0.2754308,0.6926800,⋯,-1.140143,2.158394,-0.5484047,2.67709,1.170611,-1.557405,0.5408274,1.897684,-0.4529088,1.266564
ENSG00000053702,-1.6130857,-3.105513,0.2147466,-1.933779,0.005893377,-1.3631962,-2.264548,-1.87042350,-2.0073650,-0.5491429,⋯,-1.604400,-1.509684,2.2313982,-2.23631,-1.642089,-1.556473,-1.1384676,-1.176498,-1.8472523,-3.047267


In [59]:
# Create a logical matrix indicating where NAs or NaNs are present
is_na_nan <- is.na(m_values) | is.nan(m_values)

# Summarize each row to check if all elements are NA or NaN
all_na_nan_rows <- apply(is_na_nan, 1, all)

# Identify row indices where all elements are TRUE
rows_with_only_na_nan <- which(all_na_nan_rows)

m_values <- m_values[!all_na_nan_rows, ]

### check data

In [124]:
identical(str_sort(rownames(m_values)), str_sort(selectedFeatures$V1[2413:4412]))

ERROR: Error in eval(expr, envir, enclos): object 'selectedFeatures' not found


In [ ]:
unique(colData(data_met)[which(colData(data_met)$barcode %in% colnames(m_values)), "shortLetterCode"])
unique(colData(data_met)[which(colData(data_met)$barcode %in% colnames(m_values)), "ajcc_pathologic_stage"])

## MOFA meth

In [102]:
sample <- colnames(m_values)

shortLetterCode <- ""
shortLetterCode <- apply(as.matrix(sample),1, function (x){data_met$shortLetterCode[which(data_met$sample %in% x)]})

ajcc_pathologic_stage <- ""
ajcc_pathologic_stage <- apply(as.matrix(sample),1, function (x){data_met$ajcc_pathologic_stage[which(data_met$sample %in% x)]})

age_at_diagnosis <- ""
age_at_diagnosis <- apply(as.matrix(sample),1, function (x){data_met$age_at_diagnosis[which(data_met$sample %in% x)]})

primary_diagnosis <- ""
primary_diagnosis <- apply(as.matrix(sample),1, function (x){data_met$primary_diagnosis[which(data_met$sample %in% x)]})

cigarettes_per_day  <- ""
cigarettes_per_day  <- apply(as.matrix(sample),1, function (x){data_met$cigarettes_per_day [which(data_met$sample %in% x)]})

gender <- ""
gender <- apply(as.matrix(sample),1, function (x){data_met$gender[which(data_met$sample %in% x)]})

ethnicity <- ""
ethnicity <- apply(as.matrix(sample),1, function (x){data_met$ethnicity[which(data_met$sample %in% x)]})

age_at_index <- ""
age_at_index <- apply(as.matrix(sample),1, function (x){data_met$age_at_index[which(data_met$sample %in% x)]})

race <- ""
race <- apply(as.matrix(sample),1, function (x){data_met$race[which(data_met$sample %in% x)]})

days_to_death <- ""
days_to_death <- apply(as.matrix(sample),1, function (x){data_met$days_to_death[which(data_met$sample %in% x)]})

methylation_samples <- data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, cigarettes_per_day, gender, ethnicity, 
                           age_at_index, race, days_to_death)
head(methylation_samples,2)

ERROR: Error in data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, : arguments imply differing number of rows: 427, 0


# MOFA

## mRNA

In [36]:
head(normalized_counts, 2)

,TCGA-FV-A3I0-01A,TCGA-DD-A3A6-01A,TCGA-BD-A3ER-01A,TCGA-CC-5261-01A,TCGA-DD-AAVZ-01A,TCGA-DD-AADN-01A,TCGA-DD-A1EB-01A,TCGA-LG-A9QD-01A,TCGA-G3-A25T-01A,TCGA-XR-A8TE-01A,⋯,TCGA-FV-A2QR-01A,TCGA-DD-AADY-01A,TCGA-BW-A5NO-01A,TCGA-CC-A7IJ-01A,TCGA-2Y-A9H8-01A,TCGA-BC-A10X-01A,TCGA-DD-A3A2-11A,TCGA-DD-A3A2-01A,TCGA-CC-A5UD-01A,TCGA-BC-A10W-11A
ENSG00000007350.17,-2.005792,3.668439,0.008955007,-0.7072536,-2.744387,-2.530052,-0.7269816,0.1496377,-2.037746,-1.049837,⋯,-2.237870,-0.04683096,0.3309628,-3.070753,-3.202897,3.337705,3.283273,0.6638941,-3.828411,3.483122
ENSG00000019169.10,3.770181,4.862798,3.655328409,0.4305947,-5.128186,5.235758,4.8880551,2.8254859,5.331767,6.245451,⋯,4.688055,1.03912578,1.7541576,-1.180819,3.241514,12.954035,16.244889,13.3355976,-1.905900,14.414473


In [37]:
sample <- colnames(normalized_counts)

shortLetterCode <- ""
shortLetterCode <- apply(as.matrix(sample),1, function (x){dataPrep$shortLetterCode[which(dataPrep$sample %in% x)]})

ajcc_pathologic_stage <- ""
ajcc_pathologic_stage <- apply(as.matrix(sample),1, function (x){dataPrep$ajcc_pathologic_stage[which(dataPrep$sample %in% x)]})

age_at_diagnosis <- ""
age_at_diagnosis <- apply(as.matrix(sample),1, function (x){dataPrep$age_at_diagnosis[which(dataPrep$sample %in% x)]})

primary_diagnosis <- ""
primary_diagnosis <- apply(as.matrix(sample),1, function (x){dataPrep$primary_diagnosis[which(dataPrep$sample %in% x)]})

gender <- ""
gender <- apply(as.matrix(sample),1, function (x){dataPrep$gender[which(dataPrep$sample %in% x)]})

ethnicity <- ""
ethnicity <- apply(as.matrix(sample),1, function (x){dataPrep$ethnicity[which(dataPrep$sample %in% x)]})

age_at_index <- ""
age_at_index <- apply(as.matrix(sample),1, function (x){dataPrep$age_at_index[which(dataPrep$sample %in% x)]})

race <- ""
race <- apply(as.matrix(sample),1, function (x){dataPrep$race[which(dataPrep$sample %in% x)]})

days_to_death <- ""
days_to_death <- apply(as.matrix(sample),1, function (x){dataPrep$days_to_death[which(dataPrep$sample %in% x)]})

mrna_samples <- data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, gender,ethnicity, 
                           age_at_index, race, days_to_death)
head(mrna_samples,2)

,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,gender,ethnicity,age_at_index,race,days_to_death
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>
1,TCGA-FV-A3I0-01A,TP,Stage II,28101,female,not hispanic or latino,76,white,NA
2,TCGA-DD-A3A6-01A,TP,Stage II,26319,female,not hispanic or latino,72,white,3258


### Add "-1" at the end of duplicate sample ids to modified_sample_ids to make integration easier.

In [38]:
# mrna_samples$modified_sample_ids <- mrna_samples$sample
# modified_ids <- which(duplicated(mrna_samples$sample))
# mrna_samples$modified_sample_ids[modified_ids] <- paste0(mrna_samples$modified_sample_ids[modified_ids], "-1")
# head(mrna_samples,2)

In [39]:
# mrna_samples[mrna_samples$sample %in% c("TCGA-BL-A13J-01A","TCGA-BL-A13I-01A","TCGA-BL-A0C8-01A"), ]

### Replace barcodes with sample ids at colnames

In [40]:
# colnames(normalized_counts) <- apply(as.matrix(colnames(normalized_counts)),1, function (x){mrna_samples$sample[which(mrna_samples$barcode %in% x)]})
# head(normalized_counts, 1)

### Biomart transcriptIDs to ensemblIDs

In [41]:
library(biomaRt)

In [42]:
ensembl <- useEnsembl(biomart = 'genes',dataset = "hsapiens_gene_ensembl",version = 102)

### Search for unique transcriptIDs to avoid repetitions of same transcriptIDs

In [43]:
# check number of rownames to be sure there are all in bm_results with ensembl_gene_id
length(rownames(normalized_counts))

[1] 150

In [44]:
searchAttributes(mart = ensembl, pattern = "ensembl")

,name,description,page
,<chr>,<chr>,<chr>
1,ensembl_gene_id,Gene stable ID,feature_page
2,ensembl_gene_id_version,Gene stable ID version,feature_page
3,ensembl_transcript_id,Transcript stable ID,feature_page
4,ensembl_transcript_id_version,Transcript stable ID version,feature_page
5,ensembl_peptide_id,Protein stable ID,feature_page
6,ensembl_peptide_id_version,Protein stable ID version,feature_page
7,ensembl_exon_id,Exon stable ID,feature_page
51,clone_based_ensembl_gene,Clone-based (Ensembl) gene ID,feature_page
52,clone_based_ensembl_transcript,Clone-based (Ensembl) transcript ID,feature_page


In [45]:
gene_ids_versions <- rownames(normalized_counts)

bm_results <- getBM(attributes = c('ensembl_gene_id', 'ensembl_gene_id_version'),
      filters = 'ensembl_gene_id_version',
      values = gene_ids_versions, 
      mart = ensembl)

In [46]:
dim(bm_results)
head(bm_results,2)

[1] 150   2

,ensembl_gene_id,ensembl_gene_id_version
,<chr>,<chr>
1,ENSG00000007350,ENSG00000007350.17
2,ENSG00000019169,ENSG00000019169.10


In [47]:
# Load the tidyverse package
library(tidyverse)

# Create the initial data frame
df <- data.frame(
  ensembl_gene_id_version = gene_ids_versions[grep("PAR_Y", gene_ids_versions)]
)

# Use regular expressions to extract the desired part and add it as a new column
df <- df %>%
  mutate(
    ensembl_gene_id = str_remove(ensembl_gene_id_version, "\\.\\d+")
  )

# Reorder the columns of df to match bm_results
df <- df[, colnames(bm_results)]

# Combine the two data frames using rbind
bm_results <- rbind(bm_results, df)

dim(bm_results)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%within%()    masks IRanges::%within%()
✖ dplyr::collapse()        masks IRanges::collapse()
✖ dplyr::combine()         masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()           masks matrixStats::count()
✖ dplyr::desc()            masks IRanges::desc()
✖ tidyr::expand()          masks S4Vectors::expand()
✖ dplyr::filter()          masks stats::filter()
✖ dplyr::first()           masks S4Vectors::first()
✖ dplyr::lag()             masks stats::lag()
✖ BiocGenerics::Position() masks ggplot2::Position(), base::Position()
✖ purrr::reduce()          masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()          masks S4Vectors::r

[1] 150   2

### Match ensembl_gene_id with the data from mapping_data by ensembl_transcript_id

In [48]:
rownames(normalized_counts) <- apply(as.matrix(rownames(normalized_counts)),1, function (x){bm_results$ensembl_gene_id[which(bm_results$ensembl_gene_id_version %in% x)]})
head(normalized_counts, 2)

,TCGA-FV-A3I0-01A,TCGA-DD-A3A6-01A,TCGA-BD-A3ER-01A,TCGA-CC-5261-01A,TCGA-DD-AAVZ-01A,TCGA-DD-AADN-01A,TCGA-DD-A1EB-01A,TCGA-LG-A9QD-01A,TCGA-G3-A25T-01A,TCGA-XR-A8TE-01A,⋯,TCGA-FV-A2QR-01A,TCGA-DD-AADY-01A,TCGA-BW-A5NO-01A,TCGA-CC-A7IJ-01A,TCGA-2Y-A9H8-01A,TCGA-BC-A10X-01A,TCGA-DD-A3A2-11A,TCGA-DD-A3A2-01A,TCGA-CC-A5UD-01A,TCGA-BC-A10W-11A
ENSG00000007350,-2.005792,3.668439,0.008955007,-0.7072536,-2.744387,-2.530052,-0.7269816,0.1496377,-2.037746,-1.049837,⋯,-2.237870,-0.04683096,0.3309628,-3.070753,-3.202897,3.337705,3.283273,0.6638941,-3.828411,3.483122
ENSG00000019169,3.770181,4.862798,3.655328409,0.4305947,-5.128186,5.235758,4.8880551,2.8254859,5.331767,6.245451,⋯,4.688055,1.03912578,1.7541576,-1.180819,3.241514,12.954035,16.244889,13.3355976,-1.905900,14.414473


### check gene names

In [61]:
test_df <- data.frame(gene_ids=gene_ids_versions)
head(test_df)

,gene_ids
,<chr>
1,ENSG00000007350.17
2,ENSG00000019169.10
3,ENSG00000043355.12
4,ENSG00000053108.17
5,ENSG00000060718.22
6,ENSG00000063015.20


In [62]:
test_df$modified_gene_ids <- test_df$gene_ids
test_df$modified_gene_ids <- gsub("\\..*$", "", test_df$modified_gene_ids)
head(test_df)

,gene_ids,modified_gene_ids
,<chr>,<chr>
1,ENSG00000007350.17,ENSG00000007350
2,ENSG00000019169.10,ENSG00000019169
3,ENSG00000043355.12,ENSG00000043355
4,ENSG00000053108.17,ENSG00000053108
5,ENSG00000060718.22,ENSG00000060718
6,ENSG00000063015.20,ENSG00000063015


In [63]:
identical(rownames(normalized_counts), test_df$modified_gene_ids)

[1] TRUE

## miRNA

In [49]:
head(normalized_counts_miR, 2)

,TCGA-FV-A3I0-01A,TCGA-ES-A2HS-01A,TCGA-UB-A7MA-01A,TCGA-DD-A4NV-01A,TCGA-T1-A6J8-01A,TCGA-DD-AADF-01A,TCGA-DD-AACP-01A,TCGA-CC-5261-01A,TCGA-DD-A1EC-11A,TCGA-DD-AADN-01A,⋯,TCGA-DD-AADY-01A,TCGA-CC-A7IH-01A,TCGA-CC-A7IJ-01A,TCGA-G3-AAV1-01A,TCGA-BC-A10X-01A,TCGA-G3-A25Z-01A,TCGA-DD-A3A2-11A,TCGA-DD-A3A2-01A,TCGA-ZS-A9CG-01A,TCGA-ZP-A9D0-01A
hsa-let-7c,12.01826,14.68580,12.07306,14.28009,13.80003,13.32884,12.14940,13.16593,14.79799,11.91559,⋯,12.00031,14.05701,14.73196,13.02025,14.86466,13.67965,14.30978,14.73234,14.37286,13.47230
hsa-mir-101-1,14.66983,16.88868,14.19257,15.60742,15.27675,15.90613,14.27829,15.17924,16.52022,14.38930,⋯,16.00567,17.03607,15.06988,16.32449,15.81956,15.52838,16.56375,15.94249,15.44887,15.38138


In [50]:
# miRcolData is the metadata that is needed to create DESeq2 object for miRNA
head(miRcolData, 2)

,sample,shortLetterCode,ajcc_pathologic_stage
,<chr>,<chr>,<chr>
1,TCGA-FV-A3I0-01A,TP,Stage II
2,TCGA-ES-A2HS-01A,TP,Stage I


In [66]:
# miRcolData$modified_sample_ids <- miRcolData$sample

# duplicated_sample_ids <- which(duplicated(miRcolData$sample))
# duplicates_sample_ids <- miRcolData$sample[duplicated_sample_ids]

# miRcolData$modified_sample_ids[duplicated_sample_ids] <- paste0(miRcolData$modified_sample_ids[duplicated_sample_ids], "-1")

# head(miRcolData, 1)

### Keep 16 first characters of barcode that are the sample id

In [67]:
# colnames(normalized_counts_miR) <- substr(colnames(normalized_counts_miR), 1, 16)   
# head(normalized_counts_miR, 1)

### Paste "-1" at the end of duplicated sample ids to make easier the integration

In [68]:
# duplicated_mirna <- which(duplicated(colnames(normalized_counts_miR)))
# duplicates <- colnames(normalized_counts_miR)[duplicated_mirna]
# colnames(normalized_counts_miR)[duplicated_mirna] <- paste0(colnames(normalized_counts_miR)[duplicated_mirna], "-1")

In [69]:
# colnames(normalized_counts_miR)[which(colnames(normalized_counts_miR) %in% duplicates)]
# colnames(normalized_counts_miR)[duplicated_mirna]

## Methylation

In [55]:
head(m_values, 2)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'm_values' not found


In [71]:
data_met <- readRDS("downloaded_data_met.rds")
data_met

class: RangedSummarizedExperiment 
dim: 485577 427 
metadata(1): data_release
assays(1): ''
rownames(485577): cg13869341 cg14008030 ... cg11478607 cg08417382
rowData names(52): address_A address_B ... MASK_extBase MASK_general
colnames(427): TCGA-BC-A110-11A-11D-A132-05
  TCGA-G3-A3CG-01A-11D-A20Z-05 ... TCGA-ZS-A9CG-01A-11D-A36Y-05
  TCGA-ZP-A9D0-01A-11D-A36Y-05
colData names(64): barcode patient ... released sample.aux

In [65]:
data_met <- data_met[, !duplicated(colData(data_met)$sample)]
data_met

class: RangedSummarizedExperiment 
dim: 485577 427 
metadata(1): data_release
assays(1): ''
rownames(485577): cg13869341 cg14008030 ... cg11478607 cg08417382
rowData names(52): address_A address_B ... MASK_extBase MASK_general
colnames(427): TCGA-BC-A110-11A-11D-A132-05
  TCGA-G3-A3CG-01A-11D-A20Z-05 ... TCGA-ZS-A9CG-01A-11D-A36Y-05
  TCGA-ZP-A9D0-01A-11D-A36Y-05
colData names(64): barcode patient ... released sample.aux

In [66]:
sample <- colnames(m_values)

shortLetterCode <- ""
shortLetterCode <- apply(as.matrix(sample),1, function (x){data_met$shortLetterCode[which(data_met$sample %in% x)]})

ajcc_pathologic_stage <- ""
ajcc_pathologic_stage <- apply(as.matrix(sample),1, function (x){data_met$ajcc_pathologic_stage[which(data_met$sample %in% x)]})

age_at_diagnosis <- ""
age_at_diagnosis <- apply(as.matrix(sample),1, function (x){data_met$age_at_diagnosis[which(data_met$sample %in% x)]})

primary_diagnosis <- ""
primary_diagnosis <- apply(as.matrix(sample),1, function (x){data_met$primary_diagnosis[which(data_met$sample %in% x)]})

gender <- ""
gender <- apply(as.matrix(sample),1, function (x){data_met$gender[which(data_met$sample %in% x)]})

ethnicity <- ""
ethnicity <- apply(as.matrix(sample),1, function (x){data_met$ethnicity[which(data_met$sample %in% x)]})

age_at_index <- ""
age_at_index <- apply(as.matrix(sample),1, function (x){data_met$age_at_index[which(data_met$sample %in% x)]})

race <- ""
race <- apply(as.matrix(sample),1, function (x){data_met$race[which(data_met$sample %in% x)]})

days_to_death <- ""
days_to_death <- apply(as.matrix(sample),1, function (x){data_met$days_to_death[which(data_met$sample %in% x)]})

methylation_samples <- data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, gender, ethnicity, 
                           age_at_index, race, days_to_death)
head(methylation_samples,2)

,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,gender,ethnicity,age_at_index,race,days_to_death
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>
1,TCGA-BC-A110-11A,NT,NA,18951,female,not hispanic or latino,51,black or african american,2116
2,TCGA-G3-A3CG-01A,TP,Stage I,29432,male,not hispanic or latino,80,white,NA


### Replace barcodes of meth matrix with sample ids

In [74]:
# colnames(m_values) <- apply(as.matrix(colnames(m_values)),1, function (x){data_met$sample[which(data_met$barcode %in% x)]})
# head(m_values, 2)

### Paste "-1" at the end of duplicated sample ids to make easier the integration

In [75]:
# duplicated_meth <- which(duplicated(colnames(m_values)))
# colnames(m_values)[duplicated_meth] <- paste0(colnames(m_values)[duplicated_meth], "-1")

### Add "-1" at the end of duplicate sample ids to modified_sample_ids to make integration easier.

In [76]:
# methylation_samples$modified_sample_ids <- methylation_samples$sample
# modified_ids <- which(duplicated(methylation_samples$sample))
# methylation_samples$modified_sample_ids[modified_ids] <- paste0(methylation_samples$modified_sample_ids[modified_ids], "-1")
# head(methylation_samples,2)

### Combine the rownames from the three matrices, keep only unique names (the duplictes have -1) and sort them 

In [77]:
# colnames_normalized_counts <- colnames(normalized_counts)
# colnames_normalized_counts_miR <- colnames(normalized_counts_miR)
# colnames_m_values <- colnames(m_values)
# all_colnames <- c(colnames_normalized_counts, colnames_normalized_counts_miR, colnames_m_values)
# unique_colnames <- unique(all_colnames)
# head(unique_colnames,10)
# sorted_colnames <- sort(unique_colnames)
# head(sorted_colnames,10)

# Input data for MOFA

In [51]:
sorted_colnames <- sort(colnames(normalized_counts))

### Create matrix with unique sorted colnames with the mRNA data

In [52]:
counts_new_matrix <- matrix(NA, nrow=nrow(normalized_counts), ncol=length(sorted_colnames))

colnames(counts_new_matrix) <- sorted_colnames
rownames(counts_new_matrix) <- rownames(normalized_counts)

cols <- colnames(normalized_counts)

counts_new_matrix[, cols] <- normalized_counts[, cols]
head(counts_new_matrix, 2)

,TCGA-2V-A95S-01A,TCGA-2Y-A9GS-01A,TCGA-2Y-A9GT-01A,TCGA-2Y-A9GU-01A,TCGA-2Y-A9GV-01A,TCGA-2Y-A9GW-01A,TCGA-2Y-A9GX-01A,TCGA-2Y-A9GY-01A,TCGA-2Y-A9GZ-01A,TCGA-2Y-A9H0-01A,⋯,TCGA-ZP-A9CY-01A,TCGA-ZP-A9CZ-01A,TCGA-ZP-A9D0-01A,TCGA-ZP-A9D1-01A,TCGA-ZP-A9D2-01A,TCGA-ZP-A9D4-01A,TCGA-ZS-A9CD-01A,TCGA-ZS-A9CE-01A,TCGA-ZS-A9CF-01A,TCGA-ZS-A9CG-01A
ENSG00000007350,-1.301180,-0.5387308,0.711054,-1.0437309,1.96153,-0.1947851,-2.225380,2.692144,0.4827497,0.2448743,⋯,-0.2072608,-0.6163695,-1.132313,0.5864413,-5.128186,-1.8782910,1.531825,-5.128186,-3.3269124,1.532218
ENSG00000019169,0.285627,3.5578394,1.855969,0.1906038,4.48738,9.1804624,7.793188,8.048852,2.0583166,0.7525740,⋯,9.7316176,2.1476497,2.487169,0.9428792,4.204457,0.9180244,1.161532,3.981847,-0.5955813,6.327982


### Create matrix with unique sorted colnames with the miRNA data

In [53]:
counts_miR_new_matrix <- matrix(NA, nrow=nrow(normalized_counts_miR), ncol=length(sorted_colnames))

colnames(counts_miR_new_matrix) <- sorted_colnames
rownames(counts_miR_new_matrix) <- rownames(normalized_counts_miR)

cols <- colnames(normalized_counts_miR)

counts_miR_new_matrix[, cols] <- normalized_counts_miR[, cols]
head(counts_miR_new_matrix, 2)

,TCGA-2V-A95S-01A,TCGA-2Y-A9GS-01A,TCGA-2Y-A9GT-01A,TCGA-2Y-A9GU-01A,TCGA-2Y-A9GV-01A,TCGA-2Y-A9GW-01A,TCGA-2Y-A9GX-01A,TCGA-2Y-A9GY-01A,TCGA-2Y-A9GZ-01A,TCGA-2Y-A9H0-01A,⋯,TCGA-ZP-A9CY-01A,TCGA-ZP-A9CZ-01A,TCGA-ZP-A9D0-01A,TCGA-ZP-A9D1-01A,TCGA-ZP-A9D2-01A,TCGA-ZP-A9D4-01A,TCGA-ZS-A9CD-01A,TCGA-ZS-A9CE-01A,TCGA-ZS-A9CF-01A,TCGA-ZS-A9CG-01A
hsa-let-7c,11.37719,12.95745,14.31110,11.48666,14.11517,13.27803,13.83881,12.25868,14.02407,12.31198,⋯,14.33410,12.84817,13.47230,13.54917,15.02185,14.53337,13.50983,14.12798,14.37910,14.37286
hsa-mir-101-1,14.37820,14.60589,16.55992,15.37240,16.18375,15.16547,16.08577,14.93467,16.70358,14.45255,⋯,16.17593,15.39347,15.38138,15.43699,14.87585,16.89195,14.76418,15.39147,17.24256,15.44887


### Create matrix with unique sorted colnames with the methylation data

In [81]:
m_values_new_matrix <- matrix(NA, nrow=nrow(m_values), ncol=length(sorted_colnames))

colnames(m_values_new_matrix) <- sorted_colnames
rownames(m_values_new_matrix) <- rownames(m_values)

cols <- colnames(m_values)

m_values_new_matrix[, cols] <- m_values[, cols]
head(m_values_new_matrix, 2)

,TCGA-2V-A95S-01A,TCGA-2Y-A9GS-01A,TCGA-2Y-A9GT-01A,TCGA-2Y-A9GU-01A,TCGA-2Y-A9GV-01A,TCGA-2Y-A9GW-01A,TCGA-2Y-A9GX-01A,TCGA-2Y-A9GY-01A,TCGA-2Y-A9GZ-01A,TCGA-2Y-A9H0-01A,⋯,TCGA-ZP-A9CY-01A,TCGA-ZP-A9CZ-01A,TCGA-ZP-A9D0-01A,TCGA-ZP-A9D1-01A,TCGA-ZP-A9D2-01A,TCGA-ZP-A9D4-01A,TCGA-ZS-A9CD-01A,TCGA-ZS-A9CE-01A,TCGA-ZS-A9CF-01A,TCGA-ZS-A9CG-01A
ENSG00000010319,-0.18946410,-0.4027204,-0.02789678,0.02546053,-0.1244901,0.08294856,-0.002678624,0.03639439,-0.7622871,-0.3492547,⋯,-0.1593288,0.1682615,0.2377414,-0.04067162,-0.04050503,0.1856574,-0.04788974,-0.1477604,-0.1902343,-0.2666402
ENSG00000092607,-0.03034351,0.1288488,0.22800753,1.38352861,-0.1843136,0.02110310,-0.302230768,0.14780413,0.2722933,1.4707687,⋯,0.1175350,0.7007715,0.5583310,0.43160258,0.59123893,0.1776085,-0.02348397,0.1759125,1.7133796,-0.1691787


In [82]:
# Add suffix for MOFA
rownames(m_values_new_matrix) <- paste0(rownames(m_values_new_matrix), "_methylation")

In [54]:
dim(counts_new_matrix)
dim(counts_miR_new_matrix)
dim(m_values_new_matrix)

[1] 150 408

[1] 150 408

ERROR: Error in eval(expr, envir, enclos): object 'm_values_new_matrix' not found


In [67]:
# Combine the dataframes and keep unique rows based on the ID column
combined_samples_df <- bind_rows(mrna_samples, methylation_samples) %>% distinct(sample, .keep_all = TRUE)

# Print the result
head(combined_samples_df, 1)

,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,gender,ethnicity,age_at_index,race,days_to_death
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>
1,TCGA-FV-A3I0-01A,TP,Stage II,28101,female,not hispanic or latino,76,white,NA


In [68]:
sample <- sorted_colnames

shortLetterCode <- ""
shortLetterCode <- apply(as.matrix(sample),1, function (x){combined_samples_df$shortLetterCode[which(combined_samples_df$sample %in% x)]})

ajcc_pathologic_stage <- ""
ajcc_pathologic_stage <- apply(as.matrix(sample),1, function (x){combined_samples_df$ajcc_pathologic_stage[which(combined_samples_df$sample %in% x)]})

age_at_diagnosis <- ""
age_at_diagnosis <- apply(as.matrix(sample),1, function (x){combined_samples_df$age_at_diagnosis[which(combined_samples_df$sample %in% x)]})

gender <- ""
gender <- apply(as.matrix(sample),1, function (x){combined_samples_df$gender[which(combined_samples_df$sample %in% x)]})

ethnicity <- ""
ethnicity <- apply(as.matrix(sample),1, function (x){combined_samples_df$ethnicity[which(combined_samples_df$sample %in% x)]})

# age_at_index <- ""
# age_at_index <- apply(as.matrix(sample),1, function (x){combined_samples_df$age_at_index[which(combined_samples_df$sample %in% x)]})

race <- ""
race <- apply(as.matrix(sample),1, function (x){combined_samples_df$race[which(combined_samples_df$sample %in% x)]})

days_to_death <- ""
days_to_death <- apply(as.matrix(sample),1, function (x){combined_samples_df$days_to_death[which(combined_samples_df$sample %in% x)]})

sorted_combined_df <- data.frame(sample, shortLetterCode, ajcc_pathologic_stage, age_at_diagnosis, gender, ethnicity, 
                           race, days_to_death)

head(sorted_combined_df, 2)

,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,gender,ethnicity,race,days_to_death
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
1,TCGA-2V-A95S-01A,TP,Stage II,NA,male,not hispanic or latino,asian,NA
2,TCGA-2Y-A9GS-01A,TP,NA,21318,male,hispanic or latino,white,724


In [69]:
identical(sorted_combined_df$sample, sorted_colnames)

[1] TRUE

# Create MOFA object for NT/TP

In [87]:
dim(counts_new_matrix)
dim(counts_miR_new_matrix)
dim(m_values_new_matrix)

# Combine preprocessed data into a list
data_list <- list(
  mRNA = counts_new_matrix,
  miRNA = counts_miR_new_matrix,
  Methylation = m_values_new_matrix
)

# groups <- sorted_combined_df$shortLetterCode
# length(groups)

# Create MOFA object
# MOFAobject <- create_mofa(data_list, groups = groups)
MOFAobject <- create_mofa(data_list)

[1] 150 408

[1] 150 408

[1] 150 408

Creating MOFA object from a list of matrices (features as rows, sample as columns)...




In [70]:
dim(counts_new_matrix)
dim(counts_miR_new_matrix)

# Combine preprocessed data into a list
data_list <- list(
  mRNA = counts_new_matrix,
  miRNA = counts_miR_new_matrix
)

# groups <- sorted_combined_df$shortLetterCode
# length(groups)

# Create MOFA object
# MOFAobject <- create_mofa(data_list, groups = groups)
MOFAobject <- create_mofa(data_list)

[1] 150 408

[1] 150 408

Creating MOFA object from a list of matrices (features as rows, sample as columns)...




In [71]:
MOFAobject

Untrained MOFA model with the following characteristics: 
 Number of views: 2 
 Views names: mRNA miRNA 
 Number of features (per view): 150 150 
 Number of groups: 1 
 Groups names: group1 
 Number of samples (per group): 408 
 

# Create MOFA object for tumor stage

In [206]:
dim(counts_new_matrix)
dim(counts_miR_new_matrix)
dim(m_values_new_matrix)

# Combine preprocessed data into a list
data_list <- list(
  mRNA = counts_new_matrix,
  miRNA = counts_miR_new_matrix,
  Methylation = m_values_new_matrix
)

groups <- sorted_combined_df$ajcc_pathologic_stage
length(groups)

# Create MOFA object
MOFAobject <- create_mofa(data_list, groups = groups)

[1] 150 408

[1] 150 408

[1] 150 408

[1] 408

Creating MOFA object from a list of matrices (features as rows, sample as columns)...




ERROR: Error in data.frame(sample = unname(unlist(foo)), group = unlist(lapply(names(foo), : arguments imply differing number of rows: 408, 376


In [71]:
MOFAobject

Untrained MOFA model with the following characteristics: 
 Number of views: 3 
 Views names: mRNA miRNA Methylation 
 Number of features (per view): 2000 412 2000 
 Number of groups: 4 
 Groups names: Stage I Stage II Stage III Stage IV 
 Number of samples (per group): 3 129 139 133 
 

# Define MOFA options

In [72]:
data_opts <- get_default_data_options(MOFAobject)
head(data_opts)
model_opts <- get_default_model_options(MOFAobject)
head(model_opts)
train_opts <- get_default_training_options(MOFAobject)
head(train_opts)

$scale_views
[1] FALSE

$scale_groups
[1] FALSE

$center_groups
[1] TRUE

$use_float32
[1] TRUE

$views
[1] "mRNA"  "miRNA"

$groups
[1] "group1"

$likelihoods
      mRNA      miRNA 
"gaussian" "gaussian" 

$num_factors
[1] 15

$spikeslab_factors
[1] FALSE

$spikeslab_weights
[1] FALSE

$ard_factors
[1] FALSE

$ard_weights
[1] TRUE

$maxiter
[1] 1000

$convergence_mode
[1] "fast"

$drop_factor_threshold
[1] -1

$verbose
[1] FALSE

$startELBO
[1] 1

$freqELBO
[1] 5

In [73]:
train_opts$convergence_mode <- "slow"
# train_opts$maxiter <- 2000
# head(train_opts)
model_opts$num_factors <- 10
head(model_opts)
# data_opts$scale_views <- TRUE
# data_opts$scale_groups <- TRUE
head(data_opts)

$likelihoods
      mRNA      miRNA 
"gaussian" "gaussian" 

$num_factors
[1] 10

$spikeslab_factors
[1] FALSE

$spikeslab_weights
[1] FALSE

$ard_factors
[1] FALSE

$ard_weights
[1] TRUE

$scale_views
[1] FALSE

$scale_groups
[1] FALSE

$center_groups
[1] TRUE

$use_float32
[1] TRUE

$views
[1] "mRNA"  "miRNA"

$groups
[1] "group1"

In [74]:
MOFAobject <- prepare_mofa(
  object = MOFAobject,
  data_options = data_opts,
  model_options = model_opts,
  training_options = train_opts
)

Checking data options...

Checking training options...

Checking model options...



In [75]:
outfile = file.path(getwd(),"mRNA_miRNA_model.hdf5")
MOFAobject.trained <- run_mofa(MOFAobject, outfile, use_basilisk=TRUE)

Connecting to the mofapy2 package using basilisk. 
    Set 'use_basilisk' to FALSE if you prefer to manually set the python binary using 'reticulate'.



# Load trained model

In [215]:
model <- load_model("/home/thlamp/tcga/data/model.hdf5")
# model <- load_model("/home/thlamp/tcga/bladder_results/modelStages.hdf5")
# model <- load_model("/home/thlamp/tcga/bladder_results/selectedModel.hdf5")
# model <- load_model("/home/thlamp/tcga/bladder_results/selectedStageModel.hdf5")

Warning message in .quality_control(object, verbose = verbose):
“Factor(s) 1, 6 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
”


In [216]:
head(samples_metadata(model), 1)

,group,sample
,<fct>,<chr>
1,NT,TCGA-BC-A10Q-11A


In [217]:
samples_metadata(model)$shortLetterCode <- ""
samples_metadata(model)$shortLetterCode <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$shortLetterCode[which(sorted_combined_df$sample %in% x)]})


samples_metadata(model)$ajcc_pathologic_stage <- ""
samples_metadata(model)$ajcc_pathologic_stage <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$ajcc_pathologic_stage[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$age_at_diagnosis <- ""
samples_metadata(model)$age_at_diagnosis <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$age_at_diagnosis[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$cigarettes_per_day <- ""
samples_metadata(model)$cigarettes_per_day <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$cigarettes_per_day[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$gender <- ""
samples_metadata(model)$gender <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$gender[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$ethnicity <- ""
samples_metadata(model)$ethnicity <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$ethnicity[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$age_at_index <- ""
samples_metadata(model)$age_at_index <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$age_at_index[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$race <- ""
samples_metadata(model)$race <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$race[which(sorted_combined_df$sample %in% x)]})

samples_metadata(model)$days_to_death <- ""
samples_metadata(model)$days_to_death <- apply(as.matrix(samples_metadata(model)$sample),1, function (x){sorted_combined_df$days_to_death[which(sorted_combined_df$sample %in% x)]})

head(samples_metadata(model), 1)

,group,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,gender,ethnicity,race,days_to_death
,<fct>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
1,NT,TCGA-BC-A10Q-11A,NT,NA,26400,female,not hispanic or latino,white,1135


In [218]:
# saveRDS(model@samples_metadata, "modelStages_metadata.rds")
# saveRDS(model@samples_metadata, "model_metadata.rds")
saveRDS(model@samples_metadata, "model_metadata.rds")
# saveRDS(model@samples_metadata, "selectedStageModel_metadata.rds")

In [219]:
samples_metadata <- readRDS("model_metadata.rds")
# samples_metadata <- readRDS("modelStages_metadata.rds")
# samples_metadata <- readRDS("selectedModel_metadata.rds")
# samples_metadata <- readRDS("selectedStageModel_metadata.rds")

In [220]:
dim(samples_metadata)

[1] 408   9

In [221]:
unique(samples_metadata$gender)

[1] "female" "male"

In [222]:
length(which(samples_metadata$group %in% 'Stage IV'))

[1] 0

In [223]:
model@samples_metadata <- samples_metadata
head(samples_metadata(model), 3)

,group,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,gender,ethnicity,race,days_to_death
,<fct>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
1,NT,TCGA-BC-A10Q-11A,NT,NA,26400,female,not hispanic or latino,white,1135
2,NT,TCGA-BC-A10R-11A,NT,NA,24274,female,not hispanic or latino,white,308
3,NT,TCGA-BC-A10T-11A,NT,NA,27944,male,not hispanic or latino,white,837


### check groups in model

In [77]:
dataPrep <- readRDS("downloaded_dataPrep.rds")
data_met <- readRDS("downloaded_data_met.rds")

In [78]:
check_data <- rbind(colData(dataPrep)[, c("sample", "shortLetterCode", "ajcc_pathologic_stage", "age_at_diagnosis",	
                                          "cigarettes_per_day",	"gender", "ethnicity", "age_at_index", "race", "days_to_death")], 
                    colData(data_met)[, c("sample", "shortLetterCode", "ajcc_pathologic_stage", "age_at_diagnosis",	
                                          "cigarettes_per_day",	"gender", "ethnicity", "age_at_index", "race", "days_to_death")])
dim(check_data)
head(check_data, 2)

[1] 870  10

DataFrame with 2 rows and 10 columns
                                       sample shortLetterCode
                                  <character>     <character>
TCGA-CU-A3KJ-01A-11R-A21D-07 TCGA-CU-A3KJ-01A              TP
TCGA-K4-A3WU-01B-11R-A23N-07 TCGA-K4-A3WU-01B              TP
                             ajcc_pathologic_stage age_at_diagnosis
                                       <character>        <integer>
TCGA-CU-A3KJ-01A-11R-A21D-07             Stage III            27692
TCGA-K4-A3WU-01B-11R-A23N-07             Stage III            32134
                             cigarettes_per_day      gender
                                      <numeric> <character>
TCGA-CU-A3KJ-01A-11R-A21D-07            3.28767        male
TCGA-K4-A3WU-01B-11R-A23N-07                 NA        male
                                          ethnicity age_at_index        race
                                        <character>    <integer> <character>
TCGA-CU-A3KJ-01A-11R-A21D-07 not hispanic or lati

In [79]:
check_data <- check_data[which(!duplicated(check_data$sample)), ]
dim(check_data)
head(check_data, 2)

[1] 438  10

DataFrame with 2 rows and 10 columns
                                       sample shortLetterCode
                                  <character>     <character>
TCGA-CU-A3KJ-01A-11R-A21D-07 TCGA-CU-A3KJ-01A              TP
TCGA-K4-A3WU-01B-11R-A23N-07 TCGA-K4-A3WU-01B              TP
                             ajcc_pathologic_stage age_at_diagnosis
                                       <character>        <integer>
TCGA-CU-A3KJ-01A-11R-A21D-07             Stage III            27692
TCGA-K4-A3WU-01B-11R-A23N-07             Stage III            32134
                             cigarettes_per_day      gender
                                      <numeric> <character>
TCGA-CU-A3KJ-01A-11R-A21D-07            3.28767        male
TCGA-K4-A3WU-01B-11R-A23N-07                 NA        male
                                          ethnicity age_at_index        race
                                        <character>    <integer> <character>
TCGA-CU-A3KJ-01A-11R-A21D-07 not hispanic or lati

In [80]:
head(samples_metadata(model), 5)

,group,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,cigarettes_per_day,gender,ethnicity,race,days_to_death
,<fct>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<int>
1,NT,TCGA-BL-A13J-11A,NT,Stage IV,23927,1.315068,male,not hispanic or latino,white,81
2,NT,TCGA-BT-A20N-11A,NT,Stage III,26456,2.739726,male,not reported,not reported,795
3,NT,TCGA-BT-A20R-11A,NT,Stage IV,28987,NA,female,not hispanic or latino,white,154
4,NT,TCGA-BT-A20U-11A,NT,Stage III,25761,NA,female,not hispanic or latino,white,455
5,NT,TCGA-BT-A20W-11A,NT,Stage II,26078,NA,male,not hispanic or latino,white,254


In [81]:
index <- c()
# length(sorted_combined_df$sample_ids)
for (x in 1:length(samples_metadata(model)$sample)) {
    index[x] <- which(check_data$sample %in% samples_metadata(model)$sample[x])}

In [82]:
identical(check_data$sample[index[x]], samples_metadata(model)$sample[x])

# it works only for NT/TP model
identical(check_data$shortLetterCode[index[x]], as.character(samples_metadata(model)$group[x]))
# identical(check_data$ajcc_pathologic_stage[index[x]], as.character(samples_metadata(model)$group[x]))
identical(check_data$shortLetterCode[index[x]], samples_metadata(model)$shortLetterCode[x])
identical(check_data$ajcc_pathologic_stage[index[x]], samples_metadata(model)$ajcc_pathologic_stage[x])
identical(check_data$age_at_diagnosis[index[x]], samples_metadata(model)$age_at_diagnosis[x])
identical(check_data$cigarettes_per_day[index[x]], samples_metadata(model)$cigarettes_per_day[x])
identical(check_data$gender[index[x]], samples_metadata(model)$gender[x])
identical(check_data$ethnicity[index[x]], samples_metadata(model)$ethnicity[x])
# identical(check_data$age_at_index[index[x]], samples_metadata(model)$age_at_index[x])
identical(check_data$race[index[x]], samples_metadata(model)$race[x])
identical(check_data$days_to_death[index[x]], samples_metadata(model)$days_to_death[x])

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

In [224]:
samples_metadata(model)[c(225,10,35,350,400,158), ]

,group,sample,shortLetterCode,ajcc_pathologic_stage,age_at_diagnosis,cigarettes_per_day,gender,ethnicity,age_at_index,race,days_to_death
,<fct>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>
225,TP,TCGA-FD-A6TG-01A,TP,Stage IV,26893,0.5479452,male,not hispanic or latino,73,white,93
10,NT,TCGA-GC-A3WC-11A,NT,Stage III,29295,0.7671233,female,not hispanic or latino,80,white,NA
35,TP,TCGA-4Z-AA83-01A,TP,Stage II,19280,NA,male,not reported,52,white,NA
350,TP,TCGA-XF-A9SJ-01A,TP,Stage IV,30362,3.0136986,male,not hispanic or latino,83,white,98
400,TP,TCGA-ZF-A9R9-01A,TP,Stage IV,21491,0.1643836,male,not hispanic or latino,58,white,864
158,TP,TCGA-E5-A4TZ-01A,TP,Stage IV,23566,4.6575342,male,not hispanic or latino,64,white,467


In [869]:
check_data[which(check_data$sample %in% "TCGA-BT-A2LB-01A"), 
           c("sample", "ajcc_pathologic_stage", "age_at_diagnosis", "cigarettes_per_day", "gender", "ethnicity", "age_at_index", "race", "days_to_death")]

DataFrame with 1 row and 9 columns
                                       sample ajcc_pathologic_stage
                                  <character>           <character>
TCGA-BT-A2LB-01A-11R-A18C-07 TCGA-BT-A2LB-01A             Stage III
                             age_at_diagnosis cigarettes_per_day      gender
                                    <integer>          <numeric> <character>
TCGA-BT-A2LB-01A-11R-A18C-07            26846                 NA      female
                                          ethnicity age_at_index        race
                                        <character>    <integer> <character>
TCGA-BT-A2LB-01A-11R-A18C-07 not hispanic or latino           73       white
                             days_to_death
                                 <integer>
TCGA-BT-A2LB-01A-11R-A18C-07           492